# Baseline

### Prepare data

In [1]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

TRAIN_PATH = 'data/train'
TEST_PATH = 'data/test'

to_tensor = transforms.Compose([transforms.ToTensor()])

train_data = ImageFolder(root=TRAIN_PATH, transform = to_tensor)
test_data = ImageFolder(root=TEST_PATH, transform = to_tensor)

### Define the convolutional neural network model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, num_classes=2):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc1 = nn.Linear(64, 128)  # 64 kanały z Global Poolingu
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = self.global_pool(x)
        x = torch.flatten(x, start_dim=1)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

### Helper function for calculating accuriacies

In [3]:
from torch import no_grad, max

def calculate_accuracy(loader, model, device):
    model.eval()
    with no_grad():
        correct = 0
        total = 0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

### Training loop

In [4]:
import torch
import numpy as np
from torch.cuda import is_available
from torch.utils.data import DataLoader, Subset
from torch.optim import Adam
from torch.amp.grad_scaler import GradScaler
from torch.amp import autocast
from torch.nn import CrossEntropyLoss
from sklearn.model_selection import KFold

device = torch.device('cuda' if is_available() else 'cpu')

print(device.type)

def training_loop(model, training_data, num_of_epochs):
    kf = KFold(n_splits=5, shuffle=True, random_state=0)

    for train_index, val_index in kf.split(np.arange(len(train_data))):
        train_loader = DataLoader(Subset(training_data, train_index), batch_size=32, shuffle=True)
        val_loader = DataLoader(Subset(training_data, val_index), batch_size=32, shuffle=True)
        
        model = model.to(device)
        criterion = CrossEntropyLoss()
        optimizer = Adam(model.parameters(), lr=0.001)
        scaler = GradScaler()

        train_losses = []
        val_accuracies = []

        for epoch in range(num_of_epochs):
            model.train()
            running_loss = 0.0
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                with autocast(device.type):
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                running_loss += loss.item()

            train_loss = running_loss / len(train_loader)
            train_losses.append(train_loss)
            val_accuracy = calculate_accuracy(val_loader, model, device)
            val_accuracies.append(val_accuracy)
            print(f'Epoch [{epoch+1}/{num_of_epochs}], Loss: {train_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

cpu


### Run training

In [5]:
cnn = CNN()
training_loop(cnn, train_data, 1)

c:\Uni\TAI\Magisterka\masters-thesis\venv\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


KeyboardInterrupt: 

### Helper method to calculate metrics

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_metrics(loader, model, device):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    return accuracy, precision, recall, f1, all_labels, all_predictions

### Run against test dataset

In [ ]:
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

accuracy, precision, recall, f1, all_labels, all_predictions = calculate_metrics(test_loader, cnn, device)

print(f'Test accuracy: {accuracy:.4f}%')
print(f'Test precision: {precision:.4f}')
print(f'Test recall: {recall:.4f}')
print(f'Test F1-Score: {f1:.4f}')

### Augment data

In [ ]:
augmentations = transforms.Compose([
  transforms.RandomRotation((0,90)),
  transforms.ColorJitter(),
  transforms.GaussianBlur(3),
  transforms.RandomHorizontalFlip(),
  transforms.RandomResizedCrop(224)])

augmented_train_data = ImageFolder(root=TRAIN_PATH, transform=augmentations)
augmented_test_data = ImageFolder(root=TEST_PATH, transform=augmentations)

SyntaxError: invalid syntax (3370681824.py, line 8)

In [ ]:
training_loop(cnn, augmented_train_data, augmented_test_data)